# Паттерн стратегия через полноправные функции

## Описание паттерна

В книге «Паттерны проектирования» паттерн Стратегия описывается следующим образом:

        Определить семейство алгоритмов, инкапсулировать каждый из
        них и сделать их взаимозаменяемыми. Стратегия позволяет заменять
        алгоритм независимо от использующих его клиентов.

![image](strategy.png)

Как следует из приведенной UML-даграммы, для реализации патерна авторы предлагают использовать все блага ООП, в частности, наследование, абстрактные классы, полиморфизм и пр. Однако из-за наличия в Python полноправных функций, этот паттерн может быть реализован и без использования столь сложных концепций. В этом на и предстоит убедиться.

## Задача

Рассмотрим Интернет-магазин со следующими правилами формирования скидок:
- заказчику, имеющему не менее 1000 баллов лояльности, предоставляется глобальная скидка 5 % на весь заказ;
- на позиции, заказанные в количестве не менее 20 единиц в одном заказе, предоставляется скидка 10 %;
- на заказы, содержащие не менее 10 различных позиций, предоставляется глобальная скидка 7 %.
- к каждому заказу может быть применена только одна из данных скидок - скидка, дающая наибольший бонус.

Также отдел аналитики нашего Интернет-магазина хочет собирать статистику по популярности использования каждой стратегии начисления скидки.

Наша задача, используя структы ниже, реализовать механизм расчета оптимальной скидки для каждого заказа и сбора статистика для отдела аналитики.

**Структуры для выполнения**:

In [5]:
from typing import Optional, Callable
from dataclasses import dataclass

In [6]:
@dataclass
class Item:
    label: str
    price: float
    amount: int

In [7]:
@dataclass
class Customer:
    customer_id: int
    username: str
    loyalty_points: int

In [8]:
class Order:
    customer: Customer
    order: list[Item]
    price: float

    def __init__(self, 
        customer: Customer,
        order: list[Item]
    ) -> None:
        self.customer = customer
        self.order = list(order)
        self.price = sum(item.price * item.amount for item in self.order)

In [9]:

def discount_for_1000_points(my_order) -> float:
    if my_order.customer.loyality_points >= 1000:
        return my_order.price * 0.05
    return 0
def discount_for_20_in_pos(my_order) -> float:
    total_discount = 0
    for element in my_order.order:
        if element.amount >= 20:
            total_discount += element.price * element.amount * 0.1
    return total_discount
def discount_for_10_poses(my_order) -> float:
    if len(my_order.order) >= 10:
        return my_order.price * 0.07
    return 0

popularity_counter = dict.fromkeys(
    [discount_for_1000_points,
    discount_for_20_in_pos,
    discount_for_10_poses],
)

def manage_discounts(my_order: Order) -> float:
    global popularity_counter
    max_discount_price, max_discount_func = 0, None
    
    for discount in popularity_counter:
        current_price = discount(my_order)
        
        if current_price > max_discount_price:
            max_discount_price = current_price
            max_discount_func = discount
    
    popularity_counter[max_discount_func] += 1
    
    return max_discount_price